<a href="https://colab.research.google.com/github/yishairasowsky/Sefaria-Project/blob/master/ML/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
 


In [ ]:
 
single_class_stem_df = pd.read_pickle('/content/drive/My Drive/Colab Datasets/single_class_stem_df.pkl')
single_class_stem_df.head()
df = single_class_stem_df

In [ ]:

# topics_df = pd.read_pickle('/content/drive/My Drive/Colab Datasets/single_class_topics_df.pkl')

# topics_df = pd.read_pickle('/root/Sefaria-Project/ML/data/stem_top_ten_single_topics_df.pkl')
# topics_df = pd.read_pickledata/stem_top_ten_single_topics_df.pklkl')
# topics_df.head()

In [ ]:
topics_list = list(topics_df.columns)[1:]
hits = {topic:topics_df[topic].sum() for topic in topics_list}

from collections import Counter
counter = Counter(hits)
top_ten = counter.most_common(10)
for topic, num in top_ten:
     print(f'{topic}: {num}')

In [ ]:
import matplotlib as plt

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 8

df = pd.DataFrame(top_ten, columns=['Topic','Occurences'])
df.plot(kind='bar', x='Topic')

In [ ]:
# topics_df.columns[:10]
top_ten_list = [item[0] for item in top_ten]
top_ten_list

In [ ]:
# top_ten_topics_df = topics_df[['En'] + top_ten_list]
top_ten_topics_df = topics_df[['En'] + top_ten_list]
top_ten_topics_df.head()

In [ ]:
top_ten_topics_df.to_pickle('/content/drive/My Drive/Colab Datasets/top_ten_topics_df.pkl')

In [ ]:
top_ten_labels_df = topics_df[top_ten_list]
top_ten_labels_df.head()

In [ ]:
top_ten_labels_df.to_pickle('/content/drive/My Drive/Colab Datasets/top_ten_labels_df.pkl')

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')


stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['one','say','said','may']
stopwords.extend(newStopWords)

for index, topic in enumerate(top_ten_list[:]):

    words = ' '.join(list(topics_df[topics_df[topic] == 1]['En']))
    wc = WordCloud(stopwords=stopwords).generate(words)
    
    plt.figure(
        figsize = (8, 8), 
        facecolor = 'w')
    plt.imshow(wc, interpolation="bilinear")
    plt.tight_layout(pad = 3)
    plt.axis("off")
    plt.title(topic)

plt.show()

In [ ]:
# df = top_ten_topics_df[pd.notnull(top_ten_topics_df['En'])]
# df = top_ten_topics_df[pd.notnull(top_ten_topics_df['En'])]

In [ ]:
df.info()

In [ ]:
col = ['En', 'Topics']
df = df[col]

In [ ]:
df.columns

In [ ]:
df.columns = ['En', 'Topics']

In [ ]:
df['category_id'] = df['Topics'].factorize()[0]
from io import StringIO
category_id_df = df[['Topics', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Topics']].values)

In [ ]:
top_ten_list

In [ ]:
# df = df.loc[df['Topics'].to_string in top_ten_list]
df = df[df.Topics.isin(top_ten_list)]

# df = df.loc[df['Topics'].isin()]
df.head()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))

# df.groupby(['Topic','category_id']).agg(lambda x:x.value_counts().index[0:3])

df.groupby('Topics').En.count().plot.bar(ylim=0)
plt.show()

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(df.En).toarray()
labels = df.category_id
features.shape

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np

N = 3
for Topic, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  # trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
  print("# '{}':".format(Topic))
  print("  . Most correlated unigrams:\n       . {}".format('\n       . '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n       . {}".format('\n       . '.join(bigrams[-N:])))
  # print("  . Most correlated trigrams:\n       . {}".format('\n       . '.join(trigrams[-N:])))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['En'], df['Topics'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
print(clf.predict(count_vect.transform(["You cannot cook milk with meat."])))


In [ ]:
print(clf.predict(count_vect.transform(["The kohen could bring the blood to the altar."])))


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score


models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [ ]:
import seaborn as sns

sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

In [ ]:
cv_df.groupby('model_name').accuracy.mean()


In [ ]:
from sklearn.model_selection import train_test_split

model = LinearSVC()

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:

from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.Topics.values, yticklabels=category_id_df.Topics.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:

from IPython.display import display

for predicted in category_id_df.category_id:
  for actual in category_id_df.category_id:
    if predicted != actual and conf_mat[actual, predicted] >= 6:
      print("'{}' predicted as '{}' : {} examples.".format(id_to_category[actual], id_to_category[predicted], conf_mat[actual, predicted]))
      display(df.loc[indices_test[(y_test == actual) & (y_pred == predicted)]][['Topics', 'En']])
      print('')

In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC()
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.Product.values, yticklabels=category_id_df.Product.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', 
                        encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(single_class_stem_df.En).toarray()
# labels = top_ten_topics_df.category_id
labels = single_class_stem_df.Topics
features.shape

In [ ]:
# from sklearn.feature_selection import chi2
# import numpy as np
# N = 2
# category_to_id = {topic:index for index,topic in enumerate(top_ten_list)}

# for topic, category_id in sorted(category_to_id.items()):
  
#   features_chi2 = chi2(features, labels == category_id)
#   indices = np.argsort(features_chi2[0])

#   feature_names = np.array(tfidf.get_feature_names())[indices]
#   unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
#   bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  
#   print("# '{}':".format(Product))
#   print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
#   print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

# X_train, X_test, y_train, y_test = train_test_split(top_ten_topics_df['En'], top_ten_labels_df, random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(single_class_stem_df['En'], single_class_stem_df['Topics'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
# import seaborn as sns

# categories = list(top_ten_labels_df.columns.values)
# sns.set(font_scale = 2)
# plt.figure(figsize=(15,8))
# # ax= sns.barplot(categories, top_ten_labels_df.iloc[1:,1:].sum().values)
# plt.title("Comments in each category", fontsize=24)
# plt.ylabel('Number of comments', fontsize=18)
# plt.xlabel('Comment Type ', fontsize=18)
# #adding the text labels
# rects = ax.patches
# labels = top_ten_labels_df.iloc[:,2:].sum().values
# for rect, label in zip(rects, labels):
#     height = rect.get_height()
#     ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=18)
# plt.show()

In [ ]:
print(clf.predict(count_vect.transform(["""
A woman ca get married in one of three ways.Two stages are kiddushin and chuppah. 
"""])))


In [ ]:
# using binary relevance
!pip install scikit-multilearn
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
# classifier = BinaryRelevance(GaussianNB())
classifier = MultinomialNB()
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []

for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))

cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()
